In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from dLoader import DataLoader, BuySell, generate_df

In [2]:
# Load data
stock_data = DataLoader('AAPL')

In [3]:
# Get a period of data for testing
data = stock_data.get_data('2018-01-01', '2019-12-31')
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,41.188156,102223600
2018-01-03,43.132500,43.637501,42.990002,43.057499,41.180992,118071600
2018-01-04,43.134998,43.367500,43.020000,43.257500,41.372272,89738400
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.843311,94640000
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.687893,82271200
...,...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,70.027313,48478800
2019-12-26,71.205002,72.495003,71.175003,72.477501,71.416672,93121200
2019-12-27,72.779999,73.492500,72.029999,72.449997,71.389572,146266000


In [5]:
class MACross:
    # Moving Average Fast Slow Crossover Strategy
    # Check Fast Moving Average against Slow Moving Average
    # Also checking Current Volumne Average is above 
    # Volumne Moving Average
    def __init__(self, fast=3, slow=6):
        self.fast=3
        self.slow=6
    
    def check_ma_above(self, df):
        ndf = self.get_ma(df)
        if ndf['MA-Above'].iloc[-1] and ndf['MA-Vol-Above'].iloc[-1]:
            return True
        return False

    def get_ma(self, df):
        df['Fast'] = df['Close'].rolling(self.fast).mean()
        df['Slow'] = df['Low'].rolling(self.slow).mean()
        df['MA-Vol'] = df['Volume'].rolling(self.slow).mean()
        df['MA-Above'] = df['Fast'] > df['Slow']
        df['MA-Vol-Above'] = df['Volume'] > df['MA-Vol']
        return df


In [89]:
class LookBackTest:
    def __init__(self, fast=3, slow=6):
        # Placeholder for Record of possible trades
        self.td = np.zeros(2)
        # MA Calculation Class
        self.mc = MACross(fast, slow)
        self.lookback = slow + 1
    
    def recur_lookback(self, data, i):
        if (i + 1) == len(data):
            print(self.td[0], self.td[1])
            print(self.td[0] / self.td[1])
            return
        self.update_trades(data, i)
        self.recur_lookback(data, i+1)
    
    def update_trades(self, data, idx):
        # Update record of trades
        date = data.index[idx]
        if self.is_ma_above(data, date):
            self.td[0] += 1
        self.td[1] += 1
    
    def is_ma_above(self, data, date):
        # Check if the Price and Volumne is above Moving Average
        lookback_data = self.lookback_data(data, date)
        return self.mc.check_ma_above(lookback_data)

    def lookback_data(self, df, current_date):
        # Getting Lookback data base on current date
        lookback_range = pd.date_range(end=current_date, periods=self.lookback, freq='B')
        return df.loc[df.index.isin(lookback_range[: -1])].copy()

In [90]:
lb_test = LookBackTest()

In [91]:
lb_test.recur_lookback(data, 7)

138.0 495.0
0.2787878787878788
